In [1]:
import json

In [2]:
dataset = 'aishell'
recog_set = ['train', 'dev', 'test']

In [3]:
for r in recog_set:
    json_name = f'./data/{dataset}_{r}/data_15best.json'
    with open(json_name, 'r') as f, open(f'./data/{dataset}_{r}/hyp_15best.trn', 'w') as hyp, open(f'./data/{dataset}_{r}/ref_15best.trn', 'w') as ref:
        j = json.load(f)
        for k in j['utts'].keys():
            for i, h in enumerate(j['utts'][k]['output']):
                # print(j['utts'][k]['output'])
                seq = [t for t in h['rec_token'].split()]
                hyp_seq = " ".join(seq[:-1])
                hyp.write( f'{hyp_seq} ({j["utts"][k]["utt2spk"].replace("-", "_")}-{k}-hyp[{i}])\n' )
                # hyp.write(hyp_seq + '(' + j["utts"][k]["utt2spk"].replace("-", "_") + '-' + 'hyp' + i + ')')

                ref_seq = h["token"]
                ref.write( f'{ref_seq} ({j["utts"][k]["utt2spk"].replace("-", "_")}-{k}-hyp[{i}])\n' )

上面code跑完 執行espnet的
```sclite -r ./ref.trn -h  ./hyp.trn -i rm -o all stdout >  ./result.txt```

In [18]:
nbest_err = []
with open(f'./data/aishell_train/nbest_result.txt', 'r') as f:
    temp_err = []
    for i, line in enumerate(f):
        if ('Score' in line):
            scores = line.split()[-4:]
            temp_err.append(scores)
            if (len(temp_err) == 10):
                nbest_err.append(temp_err)
                temp_err = []

In [19]:
len(nbest_err)

120098

In [20]:
nbest_token = []
nbest_score = []
ref_text = []
with open('./data/aishell_train/data.json', 'r') as f:
    j = json.load(f)
    for k in j['utts'].keys():
        token = []
        score = []
        for i, h in enumerate(j['utts'][k]['output']):
            token.append(h["rec_text"])
            score.append(h['score'])
        nbest_token.append(token)
        nbest_score.append(score)
        ref_text.append(j['utts'][k]['output'][0]['text'])

print(len(ref_text))

120098


In [21]:
dataset = []
for token, score, err, ref in zip(nbest_token, nbest_score, nbest_err, ref_text):
    temp_dict = dict()
    temp_dict['token'] = ['[CLS]' + t[:-5] + '[SEP]' for  t  in token]
    temp_dict['score'] = score
    temp_dict['err'] =  [[int(s) for s in sc] for sc in err]
    temp_dict['ref'] = '[CLS]' + ref + '[SEP]'
    dataset.append(temp_dict)
print(len(dataset))

120098


In [22]:
with open("./data/aishell_train/dataset.json", 'w') as f:
    json.dump(dataset, f, ensure_ascii=False)

## Result trn

In [2]:
dataset = 'aishell'
recog_set = ['dev', 'test']

In [5]:
training = 'MD'

In [4]:
for r in recog_set:
    json_name = f'./data/{dataset}_{r}/nBestFusionNet/rescore_data.json'
    with open(json_name, 'r') as f, open(f'./data/{dataset}_{r}/nBestFusionNet/hyp_rescore.trn', 'w') as hyp, open(f'./data/{dataset}_{r}/nBestFusionNet/ref_rescore.trn', 'w') as ref:
        j = json.load(f)
        for k in j['utts'].keys():
                # print(j['utts'][k]['output'])
            hyp_seq = j['utts'][k]['output']['rec_token']
            hyp.write( f'{hyp_seq} ({k})\n' )
            # hyp.write(hyp_seq + '(' + j["utts"][k]["utt2spk"].replace("-", "_") + '-' + 'hyp' + i + ')')
            ref_seq = j['utts'][k]['output']['text_token']
            ref.write( f'{ref_seq} ({k})\n' )

In [4]:
import matplotlib.pyplot as plt


In [ ]:
log = f'./log/MD_train.log'
train_loss = []
dev_loss = []
epoch = []
step = []

with open(log, 'r') as f:
    for line in f:
        if (line.find('training loss') > 0):
            